In [22]:
import os
import numpy as np
import pandas as pd

In [23]:
def long_short_strategy(price, prediction):
    trades = []
    position = None
    entry_price = 0
    exit_price = 0
    entry_date = None
    exit_date = None
    duration = 0
    investment_value = 1000
    quantity = 0
    return_pct = 0
    pnl = 0
    
    interest_rate_daily = 0.1 / 100  # Assuming 0.1% daily interest rate
    transaction_fee_pct = 0.5 / 100  # Assuming 0.5% transaction fee
    tax_rate = 30 / 100  # Assuming 30% tax rate

    for i in range(len(price)):
        current_price = price[i]

        # Enter or switch to a long position
        if prediction[i] == 1:
            if position == "short":  # Cover short position
                exit_price = current_price
                exit_date = i
                pnl = (entry_price - exit_price) * quantity * (1 - transaction_fee_pct)
                interest = interest_rate_daily * duration * entry_price * quantity  # Calculate interest on borrowed money
                pnl -= interest  # Deduct interest from PnL
                tax = max(0, pnl) * tax_rate  # Calculate tax if pnl is positive
                investment_value += pnl - tax  # Update investment_value after tax
                return_pct_raw = pnl / (quantity * entry_price) * 100  # Calculate return %
                return_pct = (pnl-tax) / (quantity * entry_price) * 100  # Calculate return %

                trades.append({"position": "short",
                               "entry_price": entry_price,
                               "exit_price": exit_price,
                               "entry_date": entry_date,
                               "exit_date": exit_date,
                               "pnl": pnl,
                               "return_pct": return_pct,
                               "return_pct_untaxed": return_pct_raw,
                               "duration": duration,
                               "investment_value": investment_value})

                duration = 0

            # Start long position
            if position is None or position == "short":
                position = "long"
                entry_price = current_price
                entry_date = i
                quantity = (investment_value / entry_price) * (1 - transaction_fee_pct)  # Deduct transaction fee from quantity bought
                duration = 1

        # Enter or switch to a short position
        elif prediction[i] == 0:
            if position == "long":  # Close long position
                exit_price = current_price
                exit_date = i
                investment_value = quantity * exit_price * (1 - transaction_fee_pct)  # Calculate new investment_value after close
                pnl = investment_value - (quantity * entry_price)  # Calculate PnL before tax
                tax = max(0, pnl) * tax_rate  # Calculate tax if pnl is positive
                investment_value -= tax   # Calculate final investment_value after tax
                return_pct_raw = pnl / (quantity * entry_price) * 100  # Calculate return % before tax
                return_pct = (pnl - tax) / (quantity * entry_price) * 100

                trades.append({"position": "long",
                               "entry_price": entry_price,
                               "exit_price": exit_price,
                               "entry_date": entry_date,
                               "exit_date": exit_date,
                               "pnl": pnl,
                               "return_pct": return_pct,
                               "return_pct_untaxed": return_pct_raw,
                               "duration": duration,
                               "investment_value": investment_value})

                duration = 0

            # Start short position
            if position is None or position == "long":
                position = "short"
                entry_price = current_price
                entry_date = i
                quantity = (investment_value / entry_price) * (1 - transaction_fee_pct)  # Deduct transaction fee from quantity shorted
                duration = 1

        else:
            if position is not None:
                duration += 1

    # Handle the last open position at the end of the trading period
    if position == "long":
        exit_price = current_price
        exit_date = i
        investment_value = quantity * exit_price * (1 - transaction_fee_pct)  # Calculate new investment_value after close
        pnl = investment_value - (quantity * entry_price)  # Calculate PnL before tax
        tax = max(0, pnl) * tax_rate  # Calculate tax if pnl is positive
        investment_value -= tax   # Calculate final investment_value after tax
        return_pct_raw = pnl / (quantity * entry_price) * 100  # Calculate return % before tax
        return_pct = (pnl - tax) / (quantity * entry_price) * 100

        trades.append({"position": "long",
                       "entry_price": entry_price,
                       "exit_price": exit_price,
                       "entry_date": entry_date,
                       "exit_date": exit_date,
                       "pnl": pnl,
                       "return_pct": return_pct,
                       "return_pct_untaxed": return_pct_raw,
                       "duration": duration,
                       "investment_value": investment_value})

    elif position == "short":
        exit_price = current_price
        exit_date = i
        pnl = (entry_price - exit_price) * quantity * (1 - transaction_fee_pct) 
        interest = interest_rate_daily * duration * entry_price * quantity  # Calculate interest on borrowed money
        pnl -= interest  # Deduct interest from PnL
        tax = max(0, pnl) * tax_rate  # Calculate tax if pnl is positive
        investment_value += pnl - tax  # Update investment_value after tax
        return_pct_raw = pnl / (quantity * entry_price) * 100  # Calculate return %
        return_pct = (pnl-tax) / (quantity * entry_price) * 100  # Calculate return %

        trades.append({"position": "short",
                       "entry_price": entry_price,
                       "exit_price": exit_price,
                       "entry_date": entry_date,
                       "exit_date": exit_date,
                       "pnl": pnl,
                       "return_pct": return_pct,
                       "return_pct_untaxed": return_pct_raw,
                       "duration": duration,
                       "investment_value": investment_value})

    return trades, investment_value

In [24]:
# Function to calculate metrics
def metrics_cal(trades, investment_value, price):
    risk_free_rate = 0.05  # Assumed annual risk-free rate
    trades_df = pd.DataFrame(trades)
    
    # Check if there are any trades
    if len(trades_df) == 0:
        return trades_df, None
    
    # Calculate additional metrics
    net_profit_loss = trades_df['pnl'].sum()
    total_return = (investment_value / 1000 - 1) * 100  # Adjusted for initial investment
    market_exposure = trades_df['duration'].sum() / len(price)
    volatility = trades_df['return_pct'].std()
    
    # Calculate annualized Sharpe Ratio
    if not trades_df['return_pct'].isnull().any() and not np.isneginf(trades_df['return_pct']).any():
        annualized_returns = (1 + trades_df['return_pct'] / 100).prod() ** (365 / len(price)) - 1
        annualized_volatility = volatility * np.sqrt(365)
        sharpe_ratio = (annualized_returns - risk_free_rate) / annualized_volatility
    else:
        sharpe_ratio = np.nan
        
    # Calculate running maximum
    running_max = np.maximum.accumulate(trades_df['investment_value'])
    # Ensure the value never drops below 1
    running_max[running_max < 1] = 1
    # Calculate drawdown
    drawdown = (trades_df['investment_value']) / running_max - 1
    # Calculate maximum drawdown
    max_drawdown = np.min(drawdown)
    
    # Calculate annual return
    years = len(price) / 365 
    annual_return = ((investment_value / 1000) ** (1 / years) - 1) * 100
   
    metrics = {
        "net_profit_loss": net_profit_loss,
        "total_return": total_return,
        "annual_return": annual_return,
        "sharpe_ratio": sharpe_ratio,
        "market_exposure": market_exposure,
        "volatility": volatility,
        "max_drawdown": max_drawdown
    }

    return trades_df, metrics

In [25]:
# Reading all CSV files and processing the data
folder_path = 'pred' 
files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Create empty DataFrame with models as rows and datasets as columns
models_datasets = {f.split("_")[0]: [] for f in files}
datasets = {f.split("_", 1)[1].rsplit('.', 1)[0]: [] for f in files}
df = pd.DataFrame(index=models_datasets.keys(), columns=datasets.keys())

# Processing each file
for file in files:
    model, dataset_with_extension = file.split("_", 1)
    dataset = dataset_with_extension.rsplit('.', 1)[0]
    
    data = pd.read_csv(os.path.join(folder_path, file))
    
    trades, investment_value = long_short_strategy(data['value'].values, data['prediction'].values)
    _, metrics = metrics_cal(trades, investment_value, data['value'].values)
    
    # Insert the annual return into the DataFrame
    if metrics is not None:
        df.at[model, dataset] = metrics['annual_return']
    else:
        df.at[model, dataset] = None

# Exporting the DataFrame as CSV
output_filename = 'backtesting_results_class.csv'
df.to_csv(output_filename)

print(f"Backtesting complete. Results saved to {output_filename}")


Backtesting complete. Results saved to backtesting_results_class.csv


In [26]:
df

,all_data,boruta_all_data,boruta_onchain_data,boruta_ta_data,onchain_data,ta_data,uni_data
gb,1358.145641,1747.2974,2183.984327,-81.832968,1109.21625,-81.155369,-39.467325
gru,333.394443,1161.942524,2065.033073,-73.162398,172.329696,-2.255976,-8.571132
lstm-cnn,-25.639519,659.539858,3437.557764,-56.520609,2.142287,-19.930111,-8.571132
lstm,-8.571132,-8.571132,-8.571132,-8.571132,-8.571132,-8.571132,-8.571132
lstnet,807.813381,601.259246,761.064768,-81.097167,457.78585,-56.961879,27.416261
rf,-15.402685,318.7551,955.571721,-76.694963,-14.953438,-77.572444,-58.812555
svm,964.647624,4249.402962,4970.113668,-8.571132,147.089444,-85.342892,-8.571132
tcn,31.300784,1291.457043,559.467184,-83.936404,504.487399,-60.812077,-54.772264
